In [ ]:
import pandas as pd
import neo4j

In [ ]:
# Created neo4j desktop dbms with password "senate"
driver = neo4j.GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "senate"))

In [ ]:

def run(query):
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            print(record)

def run_many(queries):
    results = []
    with driver.session() as session:
        for query in queries:
            results.append(session.run(query))
    return results

In [ ]:
run("""
MATCH (s: Senator)
DELETE s
""")

In [ ]:
def create_node(nodename, node_attrs):
    attr_list = []
    for key, value in node_attrs.items():
        attr_list.append(f'{key}: "{value}"')
    all_attrs = ", ".join(attr_list)
    return f"""
    CREATE (n:{nodename} {{{all_attrs}}})
    """
        

In [ ]:
senators = pd.read_csv("S117_members.csv")

code_to_party = {100: "democrat", 200: "republican"}
def create_senator(senator):
    name = senator["bioname"]
    node_attrs = {
        "lastname": name.split(",")[0].lower(),
        "firstname": name.split(",")[1].split()[0].lower(),
        "party": code_to_party.get(senator["party_code"], "other"),
        "state": senator["state_abbrev"],
        "icpsr": senator["icpsr"]

    }
    return create_node("Senator", node_attrs)

queries = senators.apply(create_senator, axis=1)
results = run_many(queries)


In [ ]:
run("""
MATCH (s: Senator {party: "other"})
RETURN s.firstname, s.lastname, s.party, s.icpsr
""")

In [ ]:

rollcalls = pd.read_csv("S117_rollcalls.csv")

def create_rollcall(rollcall):
    node_attrs = {
        "rollnumber": rollcall["rollnumber"],
        "date": rollcall["date"],
        "yea_count": rollcall["yea_count"],
        "nea_count": rollcall["nay_count"],
        "result": rollcall["vote_result"],
        "desc": rollcall["dtl_desc"]
    }
    return create_node("Rollcall", node_attrs)

queries = rollcalls.apply(create_rollcall, axis=1)
results = run_many(queries)